In [1]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

C:\ProgramData\Anaconda3\Lib\site-packages


In [10]:
a=dict()

In [53]:
api_url = "http://192.168.83.129:8080"
api_token = "935050ad-d55e-44dc-b658-abb960c0e175"

# OpenCTI initialization
opencti_api_client = OpenCTIApiClient(api_url, api_token)


INFO:root:Listing Threat-Actors with filters null.


In [58]:
class entity_insertion:
    
    stix_domain=dict()
        

    def __init__(self,report_path):
    
        self.report_path = report_path
    
    
    def add_mark_def(self,row,stix_id):
        mark      =row['marking_definitions']
        all_marks =opencti_api_client.marking_definition.list()
        for i in all_marks:
            if mark in i['definition'].lower():
                mark_code= i['id']
                opencti_api_client.stix_entity.add_marking_definition(id=stix_id,marking_definition_id=mark_code)  
     
    def create_threat_actor(self,report):
            
            threat_actor_details = pd.read_excel(self.report_path,sheet_name = 'threat_actor')
            threat_actor_details = threat_actor_details.where(threat_actor_details.notnull(), None)
            
            for index,row in threat_actor_details.iterrows():
            
                
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                else:
                    aliases = None
                    
                
                threat_actor = opencti_api_client.threat_actor.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                goal = row['goal'] ,
                                                sophistication = row['sophistication'] ,
                                                resource_level = row['resource_level'] ,
                                                primary_motivation = row['primary_motivation'] ,
                                                secondary_motivation = row['secondary_motivation'] ,
                                                personal_motivation = row['personal_motivation'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                #createdByRef = by,
                                                update = False
                                                 )
                    
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=threat_actor['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = threat_actor['id'] )       
                stix_domain.update ({row['name'].strip() : threat_actor['id']})
                        
    def create_attack_pattern(self,report):           #attack_pattern insertion    
            
            attack_pattern_details = pd.read_excel(self.report_path,sheet_name = 'attack_pattern')
            attack_pattern_details = attack_pattern_details.where(attack_pattern_details.notnull(), None)
            
            for index,row in attack_pattern_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                else:
                    aliases = None
                    
                if row['platform'] is not None:
                    platforms =row['platform'].split(',')
                else:
                    platforms = None
                    
                print(row['required_permission'])   
                
                exist = opencti_api_client.attack_pattern.list ( filters=[{'key' : 'name' , 'values' : row['name']}])

                
                if (len(exist) < 0):
                    attack_pattern = opencti_api_client.attack_pattern.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases,
                                                platform  = platforms ,
                                                required_permission = row['required_permission'] ,
                                                external_id  = row['external_id'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                #createdByRef = by,
                                                )
                else:
                     attack_pattern = opencti_api_client.attack_pattern.create(
                                                name = row['name'],
                                                description = row['description'],
                                                alias = aliases ,
                                                platform  = platforms ,
                                                required_permission = row['required_permission'] ,
                                                external_id  = row['external_id'] ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=attack_pattern['id'],row=row)
                        
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = attack_pattern['id'] )  
                stix_domain.update ({row['name'].strip() : attack_pattern['id']})      
                        
    def create_malware(self,report):           #malware insertion    
            
            malware_details = pd.read_excel(self.report_path,sheet_name = 'malware')
            malware_details = malware_details.where(malware_details.notnull(), None)
            
            for index,row in malware_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.malware.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) < 0):
                    malware = opencti_api_client.malware.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                #createdByRef = by,
                                                )
                else:
                     malware = opencti_api_client.malware.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=malware['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = malware['id'] )
                stix_domain.update ({row['name'].strip() : malware['id']}) 
                
    def create_tool(self,report):           #malware insertion    
            
            tool_details = pd.read_excel(self.report_path,sheet_name = 'tool')
            tool_details = tool_details.where(tool_details.notnull(), None)
            
            for index,row in tool_details.iterrows():
            
                if row['alias'] is not None:
                    aliases =row['alias'].split(',')
                    print(aliases)
                else:
                    aliases = None
                
                exist = opencti_api_client.tool.list ( filters=[{'key' : 'name' , 'values' : row['name']}])
            
                if (len(exist) > 0):
                    tool = opencti_api_client.tool.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                #createdByRef = by,
                                                )
                else:
                     tool = opencti_api_client.tool.create(
                                                name = row['name'].strip(),
                                                description = row['description'],
                                                alias = aliases ,
                                                modified = datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00") ,
                                                update = True,
                                                 )
                if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=tool['id'],row=row)
                        
                opencti_api_client.report.add_stix_entity(id=report,entity_id = tool['id'] )
                stix_domain.update ({row['name'].strip() : tool['id']})
        
    def create_report(self):

        report_details = pd.read_excel(self.report_path,sheet_name = 'report')
        report_details = report_details.where(report_details.notnull(), None)
        
        # report creation
        for index,row in report_details.iterrows():
#             report_by = opencti_api_client.identity.list(filters=[{'key':'name','values': row['created_by_ref'] }])
           
#             if len(report_by) == 0 : 
#                 warnings.warn("Since " + row['created_by_ref'] + " is not inserted in opencti.Rhymtechnologies is taken as default")
#                 report_by = organization = opencti_api_client.identity.create(
#                                             type='Organization',
#                                             name="Rhymtechnologies",
#                                             alias=["Rhymtech"],
#                                             description="A young and innovative company formed with a vision to create world leading Cyber Security products and services. The promoters are a group of highly successful professionals with diverse backgrounds across IT Delivery, Sales and Consulting, Cyber Security, Artificial Intelligence, Machine Learning and Data Sciences. Our group of consultants led by some of the best minds in the industry have successfully provided strategic and value-add services for multiple large organizations globally.",
#                                             )
#             else:
#                 report_by = report_by[0]
                
           # report_by = {'id': None}
                
            
            print(row ['source_confidence_level'])
            
            report = opencti_api_client.report.create(
                                                            name=row['name'],
                                                            description= row['description'],
                                                            report_class="Internal Report",
                                                            published=datetime.date.today().strftime("%Y-%m-%dT%H:%M:%S+00:00"),
                                                            #createdByRef= report_by['id'],
                                                            object_status = row['object_status'],
                                                            source_confidence_level = row ['source_confidence_level'],
                                                            update = True,
                                                            )
            if row['marking_definitions'] is not None:
                        self.add_mark_def(stix_id=threat_actor['id'],row=row)
            print("one_completed")
            
            
            self.create_threat_actor(report = report['id'])  
            self.create_attack_pattern( report =report ['id'])
            self.create_malware (report = report ['id'])
            self.create_tool (report = report ['id'])
    
                 
                
            

In [59]:
opencti_api_client.get_identity( id =  'ce15efb8-d60d-4fd4-b68d-e17dabf6310f')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method get_identity. (Replaced by the Identity class in pycti) -- Deprecated since version 2.1.0.
  """Entry point for launching an IPython kernel.
INFO:root:Reading Identity {ce15efb8-d60d-4fd4-b68d-e17dabf6310f}.


In [61]:
en=entity_insertion(report_path=r"D:\Threat_intel_services\srikanth\fakeapt12.xlsx")
en.create_report()

INFO:root:Listing Reports with filters [{"key": "name", "values": ["fakeapt12"]}, {"key": "published_day", "values": ["2020-01-29"]}].


1.0


INFO:root:Listing Stix-Domain-Entities with filters [{"key": "name", "values": ["fakeapt12"]}].
INFO:root:{"errors":[{"message":"Cannot query field \"observableRefs\" on type \"Incident\".","locations":[{"line":126,"column":17}],"extensions":{"code":"GRAPHQL_VALIDATION_FAILED"}}]}



one_completed


TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(report_by['id'])